In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('LR').getOrCreate()

In [3]:
df=spark.read.csv('/FileStore/tables/cruise_ship_info.csv',header=True,inferSchema=True)

In [4]:
df.show(2)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+
Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 2 rows

In [5]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [6]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='Cruise_cat')
indes=indexer.fit(df).transform(df)

In [9]:
indes.show(2)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_cat|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
only showing top 2 rows

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
indes.columns

Out[42]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

In [12]:
assembler=VectorAssembler(inputCols=['Age','Tonnage', 'passengers', 'length', 'cabins', 'passenger_density','Cruise_cat'],outputCol='features')

In [13]:
output=assembler.transform(indes)

In [14]:
output.select(['features','Crew']).show()

+--------------------+----+
 features|Crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [15]:
final=output.select(['features','Crew'])

In [16]:
train_data,test_data=final.randomSplit([0.75,0.25])

In [17]:
train_data.describe().show()

+-------+------------------+
summary| Crew|
+-------+------------------+
 count| 122|
 mean| 7.667295081967226|
 stddev|3.5978331488206163|
 min| 0.59|
 max| 21.0|
+-------+------------------+

In [18]:
test_data.describe().show()

+-------+------------------+
summary| Crew|
+-------+------------------+
 count| 36|
 mean| 8.224166666666667|
 stddev|3.1721586836546676|
 min| 2.87|
 max| 19.1|
+-------+------------------+

In [19]:
from pyspark.ml.regression import LinearRegression

In [20]:
ship_lr=LinearRegression(labelCol='Crew')

In [21]:
final_train=ship_lr.fit(train_data)

In [22]:
test_result=final_train.evaluate(test_data)

In [23]:
test_result.r2

Out[57]: 0.8166999519444783

In [24]:
test_result.rootMeanSquaredError

Out[58]: 1.3391183630803782

In [25]:
test_result.meanAbsoluteError

Out[59]: 0.6753772917579058

In [26]:
from pyspark.sql.functions import corr

In [27]:
df.select(corr('Crew','passengers')).show()

+----------------------+
corr(Crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [28]:
df.select(corr('Crew','cabins')).show()

+------------------+
corr(Crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+